In [1]:
import pickle
import numpy as np
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def save_lists_to_file(path):
    # Open the pickle file in binary write mode
    with open(path, 'rb') as f:
    # Load each list from the file
        NE = pickle.load(f)
        EEG_segments = pickle.load(f)
        Classes = pickle.load(f)

    return NE, EEG_segments, Classes

In [3]:
def padding_x_y(EEG_segments, Classes, named_entity_list):
    X = []
    y = []
    NE = []
    for i in range(len(EEG_segments)):
        named_entity = named_entity_list[i]
        label = Classes[i][0]
        #print(label)
        EEG_list = EEG_segments[i]
        for EEG in EEG_list:
            if EEG != []:
                X.append(EEG)
                y.append(label)
                NE.append(named_entity)
    max_seq_length = max([len(x) for x in X])
    #paddding
    for i in range(len(X)):
        padding_count = max_seq_length - len(X[i])
        #print(padding_count)
        for j in range(padding_count):
            X[i].append(np.zeros((105,8)))

    return X, y, NE


In [4]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=50, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")


    return word_embeddings

In [5]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(y):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(y)

    y_categorical = to_categorical(encoded_labels)

    return y_categorical

In [6]:
def reshape_data(X):
    #reshape the data to 840
    new_list = []
    for i in range(len(X)):
        array_list = X_train_numpy[i]
        arrays_list_reshaped = [arr.reshape(-1) for arr in array_list]
        new_list.append(arrays_list_reshaped)

    new_list = np.array(new_list)
    return new_list

In [7]:
#read in train and test data

train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [8]:

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [9]:
#Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)

In [10]:
train_NE, train_EEG_segments, train_Classes = save_lists_to_file(train_path)
test_NE, test_EEG_segments, test_Classes = save_lists_to_file(test_path)

In [10]:

#padding
X_train, y_train, NE_list = padding_x_y(train_EEG_segments, train_Classes, train_NE)
X_train_numpy = np.array(X_train)
X_train_numpy = reshape_data(X_train_numpy)
y_train_categorical = encode_labels(y_train)

X_test, y_test, NE_list_test     = padding_x_y(test_EEG_segments, test_Classes, test_NE)
X_test_numpy = np.array(X_test)
X_test_numpy = reshape_data(X_test_numpy)
y_test_categorical = encode_labels(y_test)

In [11]:
pairs = list(zip(NE_list, y_train_categorical))

In [13]:
import random

sample_size = int(0.1 * len(NE_list))

# Randomly sample 10% of the words
sampled_words = random.sample(pairs, sample_size)

NE_sampled, y_train_sampled = zip(*sampled_words)


In [14]:
sample_size

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [15]:
NE_sampled

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [264]:
y_train_sampled

(array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 

In [197]:
flat_list = [word for sublist in NE_list for word in sublist]
len(flat_list)

25865

In [198]:
word_embeddings = create_word_label_embeddings(flat_list)

Number of word embeddings: 683


In [223]:
def NE_embeddings(NE_list, word_embeddings):
    NE_list_embeddings = []
    for i in range(len(NE_list)):
        max_length = 7
        words = NE_list[i]
        words_embedding_list = []
        for word in words:
            if word in word_embeddings:
                embedding_word = word_embeddings[word]
                words_embedding_list.append(embedding_word)
            else:
                print(word)
        #padding
        if len(words_embedding_list) < max_length:
            for i in range(max_length - len(words_embedding_list)):
                words_embedding_list.append(np.zeros(50))
        NE_list_embeddings.append(words_embedding_list)

    return NE_list_embeddings

In [224]:
NE_list_embeddings = NE_embeddings(NE_list, word_embeddings)

In [146]:
len(X_train_numpy)

21875

In [167]:
len(NE_list)

21875

In [169]:
NE_list = np.array(NE_list)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21875,) + inhomogeneous part.

In [147]:
print(X_train_numpy.shape)
print(y_train_categorical.shape)

(21875, 7, 840)
(21875, 3)


BLSTM pytorch implementation

In [225]:
import torch
from torch.utils.data import DataLoader, TensorDataset


associated_words_tensor = torch.tensor(NE_list_embeddings, dtype=torch.float32)

In [226]:


# Assuming x_train and y_train are your input data and target labels, respectively

# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(X_train_numpy, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_categorical, dtype=torch.float32)  # Assuming your labels are integers

# Create a custom dataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor, associated_words_tensor)

# Define batch size
batch_size = 32  # Adjust according to your preference

# Create the train loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [149]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [150]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [151]:
# Define the BLSTM classifier model
class BLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(BLSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for bidirectional

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # *2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out

In [152]:
# Define model parameters
input_size = 840
hidden_size = 64
num_layers = 2
num_classes = 3

In [153]:

# Instantiate the model
model = BLSTMClassifier(input_size, hidden_size, num_layers, num_classes)
model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [254]:
num_epochs = 10

In [155]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)


        optimizer.zero_grad()

        outputs = model(batch_x)

        # Convert class probabilities to class indices
        _, predicted = torch.max(outputs, 1)

        loss = criterion(outputs, batch_y.squeeze())  # Ensure target tensor is Long type
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')


Epoch [1/10], Loss: 0.9415


KeyboardInterrupt: 

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'blstm_model.pth')

# Load the saved model
loaded_model = BLSTMClassifier(input_size, hidden_size, num_layers, num_classes)
loaded_model.load_state_dict(torch.load('blstm_model.pth'))
loaded_model.eval()  # Switch to evaluation mode

# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    for batch_x, batch_y in test_loader:  # Assuming you have a test DataLoader
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_x = batch_x.squeeze(3)
        outputs = loaded_model(batch_x)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    print('Accuracy of the model on the test set: {}%'.format(100 * correct / total))